# Estimating Settlement Carbon Stocks using Land Cover and Land Use Data

Created by: Derek Robinson @UWaterloo in collaboration with Douglas MacDonald, Lyna Lapointe-Elmrabti, Cameron Samonson, and Meghan Bishoff at Environment and Climate Change Canada<br>
Last Updated: February 28, 2022

### Learning Objectives

In this Jupyter Notebook you will learn or review 
<ul>
    <li> an approach for carbon stock estimation based on a combination of IPCC and Environment and Climate Change Canada data and methods; </li>
    <li> gain more experience with R; and </li>
    <li> estimate carbon stocks for different land cover types that have different amounts and management activities on different land uses.</li>  
</ul>

### Assignment Completion

The assignment will be completed as a quiz provided on LEARN that spans multiple days. To complete the quiz/assignment you may need to do the following:

<ul>
    <li> work through this notebook, </li>
    <li> copy results from your notebook into the LEARN input boxes, </li>
    <li> take screen captures of your notebook output and submit them via LEARN, or </li>
    <li> change values in the Jupyter Notebook and re-execute cells/code to obtain new results. </li>
</ul>

To advance to the next text cell or execute the code in a cell hold <font style="font-family:'Courier New'"><mark font-family="Courier New" style="background-color: #F5F5F5">Shift</mark></font> and press <font style="font-family:'Courier New'"><mark font-family="Courier New" style="background-color: #F5F5F5">Enter</mark></font>. If there is a method for which you would like to know more about its parameters or how it works then you can place your cursor on the method and press <font style="font-family:'Courier New'"><mark font-family="Courier New" style="background-color: #F5F5F5">Shift + Tab</mark></font> and a help box will appear that you can expand and scroll through.

<i> <b>While you may discuss course content with your classmates, you are to complete the assignment individually.</b> </i>

### Overview

The storage and flux of terrestrial carbon is spatially heterogeneous partly due to how humans use (i.e., `land use`), manage, and change what resides on the landscape (i.e., `land cover`). To better understand the extent of land use and land cover change, extensive effort has gone into mapping historical and global land use and land cover. At least 10 different global land cover datasets have been generated. However, the spatial resolution of these global data is too coarse (30m – 1km) to accurately capture the quantity and location of land cover in highly fragmented and human dominated Settlement areas.

`Settlement` areas are differentially defined by country, but can be generalized as all developed land not included in other `Land Use, Land Use Change and Forestry` (LULUCF) sector categories reported to the United Nations Framework Convention on Climate Change (UNFCCC). Within the LULUCF sector, Settlements are of particular interest for carbon reporting because 1) urbanization projections estimate that 68% of the global population and greater than 87% of the North American population will reside within urban settlement areas by 2050 (UN 2018) and it is necessary to engage this population if we are to mitigate the impacts of climate change, 2) conversions from other land uses to Settlement are typically unidirectional and consist of significant manipulation of the landscape, and 3) Settlements have the potential to remove CO<sub>2</sub> from the atmosphere or equally result in emissions, depending on the type and severity of disturbance.

While there is a wealth of research linking land use and the carbon cycle (e.g., Brown et al. 2013), there remains a paucity of research, particularly the classification of land cover required for the quantification of the change in carbon on the landscape when land is converted to Settlement or to estimate carbon emissions or removals within Settlement areas. 

The content you will work through in this notebook builds on the following efforts to map land cover, quantify carbon stocks and change, and model urban growth based on property parcel data:

* Land-cover fragmentation and configuration of ownership parcels in an exurban landscape<br>
https://link.springer.com/article/10.1007/s11252-011-0205-4

* Quantifying spatial–temporal change in land-cover and carbon storage among exurban residential parcels<br>
https://link.springer.com/article/10.1007/s10980-013-9963-0

* Comparison of Statistical Approaches for Modelling Land-Use Change<br>
https://www.mdpi.com/2073-445X/7/4/144


To more accurately quantify carbon stocks in Settlement areas, high spatial resolution imagery (20 cm) were acquired from the Province of Ontario, Canada. The imagery were classified using object-based image analysis into one of the following `seven land cover classes: Grass, Coniferous, Deciduous, Cropland and Soil, Water, Impervious, Shadow`. Parcel data were manually interpreted using the same imagery to assign each parcel to one of the following `eleven land uses: Low Density Residential, Medium Density Residential, High Density Residential, Commercial, Industrial, Institutions, Transportation, Protected Areas, Agriculture, Water, and Under Development`. Then the land cover and land use data were intersected to derive land cover within each property parcel. The subsequent estimates of carbon are based on a combination of land use (Activity Data) and amount of area in different land covers. These estimates are then summarized by land-use type and property size within this notebook.

An example area of the classified imagery is provided below.

<img src="LC_Classification.png" alt="drawing" width="600"/>

# Tree Cover Carbon

In the presented approach, all types of trees are lumped together and represented as tree cover. Assessment of Settlement tree cover across multiple locations in Canada has been conducted by Steenberg et al. (in review), which currently has the following title: "An aupdated approach for assessing Canada's urban forest carbon storage and sequestration". Within this document, the average urban tree-cover carbon storage for the Mixedwood Plains ecozone in Canada is calculated as 57.8 t ha<sup>-1</sup>. 

In [ ]:
C_trees_t_ha <- 57.8

For comparative purposes, it is good to make sure we understand how to work among different units. We can derive kg ha<sup>-1</sup> by multiplying our value of t ha<sup>-1</sup> by 1000 (1000 kg = 1 Mt). Then we can derive kg C m<sup>-2</sup> by dividing by 10000 since 10,000 m<sup>2</sup> = 1 ha

In [ ]:
C_trees_kg_ha <- C_trees_t_ha * 1000

In [ ]:
C_trees_kg_m2 <- C_trees_kg_ha/10000

Our estimate of carbon stock in tree cover per metre square is:

In [ ]:
C_trees_kg_m2

# Derivation of soil carbon estimates by land use

While the above approach to tree cover carbon estimation embeds the work and assumptions of Steenberg et al. (in review), our approach to estimate soil carbon is more involved.

Using an IPCC Tier 1 approach, soil carbon in different land uses is calculated relative to initial values in cropland using the following equation:

<i>C<sub>Soils_LC</sub> = C<sub>Cropland</sub> * IPCC<sub>LC</sub> * (1 - Prop<sub>Modified</sub>) + (0.5 * C<sub>Cropland</sub>) * IPCC<sub>LC</sub> * Prop<sub>Modified</sub></i><br><br>

whereby:<br>
<ul style="list-style-type:none">
    <li><i>C<sub>Soils_LC</sub></i> = Total soil carbon for a land cover in Mg C ha<sup>-1</sup>, which comprises both areas where soils have been modified by landscaping (i.e., soil extraction, grading and 15 cm of topsoil replaced) and unmodified (areas maintaining at least 30 cm depth of soil and conditions remnant of cropland land use)</li><br>
    <li><i>C<sub>Cropland</sub></i> = Initial carbon stocks for cropland in the region were based on average soil carbon derived from CanSIS databases for medium textured cropland soils in the KCW region based on soil carbon in the upper 30 cm of the soil horizon, equivalent to 70 Mg C ha<sup>-1</sup></li><br>
    <li><i>IPCC<sub>LC</sub></i> = the factor multiplied against cropland soil carbon to determine soil carbon in different land covers (LCs). Acquired from IPCC 2006, Vol. 4. The Settlements chapter (Chapter 8), in Section 8.3.3.2 Mineral Soils, points to Grassland chapter (Chapter 6, Table 6.2) for the IPCC relative stock change factors for grassland management. We use a Land Use Factor (F<sub>LU</sub>) = 1 and the Management Factor (F<sub>MG</sub>) for Improved Grassland for the Temperate/Boreal climate region = 1.14 in our estimate of carbon in turfgrass in Settlement areas. A value of 1.1 is used for tree cover, which is derived from Table 5.5 (Chapter 5) for No-Till (F<sub>MG</sub>) in a Dry Temperate/Boreal climate region.</li><br>
        <li><i>Prop<sub>Modified</sub></i> = proportion of the land use area assumed to be landscaped and modified such that 15 cm of topsoil has been applied over fill with only 50% of <i>C<sub>Cropland</sub></i>. The proportion modified changes by land-use type.</li><br>
</ul>

CANSIS Database: http://sis.agr.gc.ca/cansis <br>
Chapter 8: https://www.ipcc-nggip.iges.or.jp/public/2006gl/pdf/4_Volume4/V4_08_Ch8_Settlements.pdf <br>
Chapter 6: https://www.ipcc-nggip.iges.or.jp/public/2006gl/pdf/4_Volume4/V4_06_Ch6_Grassland.pdf

Using the above values, variables are created for each


In [ ]:
C_cropland <- 70

In [ ]:
IPCC_soils_grass <- 1.14

Table 6.2<br>
<img src="Table6.2.png" alt="drawing" width="800"/><br>
https://www.ipcc-nggip.iges.or.jp/public/2006gl/pdf/4_Volume4/V4_06_Ch6_Grassland.pdf

In [ ]:
IPCC_soils_trees <- 1.1

Table 5.5 <br>
<img src="Table5.5.png" alt="drawing" width="800"/><br>
https://www.ipcc-nggip.iges.or.jp/public/2006gl/pdf/4_Volume4/V4_05_Ch5_Cropland.pdf

## Residential Land Uses
Using the above equation for <i>C<sub>Soils</sub></i> and assumptions about the proportion of residential land-use types that have modified soils, we can estimate the total soil carbon associated with turfgrass in residential land uses. Soils under Grass and Tree cover were divided into two categories, either landscaped or unmodified, where the landscaped sites are assumed to have 15 cm of topsoil applied over fill with 50% of the carbon content observed for croplands. The extent of landscaped disturbance from construction on parcels was assumed to reach a maximum of 1 acre or 0.405 ha. Given our parcel size distribution for residential land uses, high and medium density parcels were assumed to be 100% landscaped and low-density residential parcels were assumed to be 10% landscaped.

Create a variable for each of these proportions of modified soils for each residential density. LDR = Low Density Residential, MDR = Medium Density Residential, HDR = High Density Residential.

In [ ]:
LDR_prop_modified <- 0.1
MDR_prop_modified <- 1.0
HDR_prop_modified <- 1.0

Place these proportions, and prior variables, into our equation for <i>C<sub>Soils_Grass</sub></i>.

In [ ]:
C_LDR_soils_grass <- C_cropland*IPCC_soils_grass*(1 - LDR_prop_modified)+(0.5*C_cropland)*IPCC_soils_grass*LDR_prop_modified
C_MDR_soils_grass <- C_cropland*IPCC_soils_grass*(1 - MDR_prop_modified)+(0.5*C_cropland)*IPCC_soils_grass*MDR_prop_modified
C_HDR_soils_grass <- C_cropland*IPCC_soils_grass*(1 - HDR_prop_modified)+(0.5*C_cropland)*IPCC_soils_grass*HDR_prop_modified

Estimates of soil C in Mg C ha<sup>-1</sup> of turfgrass for each residential land-use type are as follows:

In [ ]:
C_LDR_soils_grass
C_MDR_soils_grass
C_HDR_soils_grass

Using the same approach as above, we can estimate soil carbon under Tree cover (Mg C ha<sup>-1</sup>) for each residential land-use type as follows:

In [ ]:
C_LDR_soils_trees <- C_cropland*IPCC_soils_trees*(1 - LDR_prop_modified)+(0.5*C_cropland)*IPCC_soils_trees*LDR_prop_modified
C_MDR_soils_trees <- C_cropland*IPCC_soils_trees*(1 - MDR_prop_modified)+(0.5*C_cropland)*IPCC_soils_trees*MDR_prop_modified
C_HDR_soils_trees <- C_cropland*IPCC_soils_trees*(1 - HDR_prop_modified)+(0.5*C_cropland)*IPCC_soils_trees*HDR_prop_modified

Estimates of soil C under Tree cover (Mg C ha<sup>-1</sup>) for each residential land-use type are:

In [ ]:
C_LDR_soils_trees
C_MDR_soils_trees
C_HDR_soils_trees

## Commercial, Industrial, Institutions, and Transportation Land Uses

Soil carbon calculations for Commercial, Industrial, Institutions, and Transportation are done in a similar manner. The <i>IPCC<sub>Grass</sub></i> value is multipled against 50% of the C<sub>Cropland</sub>, with the assumption of 100% modified soil for each of the these land-use types. 

Set the proprtion modified

In [ ]:
comm_prop_modified <- 1.0

Generate the estimated soil C for commercial

In [ ]:
C_comm_soils_grass <- C_cropland*IPCC_soils_grass*(1 - comm_prop_modified)+(0.5*C_cropland)*IPCC_soils_grass*comm_prop_modified

Set the same value for other land use types:

In [ ]:
C_ind_soils_grass <- C_comm_soils_grass
C_inst_soils_grass <- C_comm_soils_grass
C_trans_soils_grass <- C_comm_soils_grass

Estimates of soil C in Mg C ha<sup>-1</sup> for commercial, industrial, institutions, and transportation are as follows: 

In [ ]:
C_comm_soils_grass
C_ind_soils_grass
C_inst_soils_grass
C_trans_soils_grass

Next we estimate the above ground carbon storage in tree cover. We set commercial, industrial, and transportation soils under tree cover to zero, and use the same equation for soils under grass for under trees for Institutions.

In [ ]:
C_comm_soils_trees <- 0
C_ind_soils_trees <- 0
C_inst_soils_trees <- C_cropland*IPCC_soils_trees*(1 - comm_prop_modified)+(0.5*C_cropland)*IPCC_soils_trees*comm_prop_modified
C_trans_soils_trees <- 0

This gives us the following soil carbon (Mg ha<sup>-1</sup> under tree cover estimates of 

In [ ]:
C_comm_soils_trees
C_ind_soils_trees
C_inst_soils_trees
C_trans_soils_trees

## Protected Areas

The calculation for soil carbon under turfgrass in protected areas follows the same assumptions used for low-density residential and therefore the Mg C ha<sup>-1</sup> is the same.

In [ ]:
C_prot_soils_grass <- C_LDR_soils_grass

Estimate of Mg C ha<sup>-1</sup> in soils in protected areas is:

In [ ]:
C_prot_soils_grass

Similarly, the calculation for soil carbon under tree cover for protected areas uses the same assumptions used for low-density residential.

In [ ]:
C_prot_soils_trees <- C_LDR_soils_trees

Giving us the following Mg C ha<sup>-1</sup>

In [ ]:
C_prot_soils_trees

## Agriculture and Under development Turfgrass

Soil carbon under agriculture was acquired by the CANSIS data as discussed above. Since the majority of conversions to Settlement in southern Ontario are from agricultural lands, we assume that areas Under Development retain the agricultural carbon values until fully transformed into a new land use.

In [ ]:
C_ag_soils_grass <- 70.0
C_und_soils_grass <- 70.0

Confirm the values

In [ ]:
C_ag_soils_grass
C_und_soils_grass

Soil under trees in agricultural land uses are assumed the same as the CANSIS data at 70 Mg C ha<sup>-1</sup>.
However, soils in Under Development land cover are assumed to be similar to low-density residential that have some landscape disturbance.

In [ ]:
C_ag_soils_trees <- 70.0
C_und_soils_trees <- C_LDR_soils_trees

Yielding a slightly higher value for soils under tree cover in Under Developed land use compared to Agricultural land use.

In [ ]:
C_ag_soils_trees
C_und_soils_trees

# Table of Soil Carbon by Land Use

Using the results from above we can make four columns of data and combine them using `cbind()` to create a dataframe of our values. First we will make a list of our land use categories. I note the variable as lulc because technically Water is a land cover.

In [ ]:
lulc_categories <- c("Low Density Residential", "Medium Density Residential", "High Density Residential", "Commercial", 
"Industrial", "Institutions", "Transportation", "Protected Areas", "Agriculture", "Water", "Under Development")

Next we will create a variable for each of our soil C estimates under turfgrass for each land use.

In [ ]:
lulc_grass_soilC <- c(C_LDR_soils_grass, C_MDR_soils_grass, C_HDR_soils_grass, C_comm_soils_grass, C_ind_soils_grass, 
                      C_inst_soils_grass, C_trans_soils_grass, C_prot_soils_grass, C_ag_soils_grass, 0, C_und_soils_grass) 

Then we create a variable holding our soil C estimates under tree cover for each land use.

In [ ]:
lulc_trees_soilC <- c(C_LDR_soils_trees, C_MDR_soils_trees, C_HDR_soils_trees, C_comm_soils_trees, C_ind_soils_trees, 
                      C_inst_soils_trees, C_trans_soils_trees, C_prot_soils_trees, C_ag_soils_trees, 0, C_und_soils_trees) 

Finally we create a variable holding our soil C estimates under cropland for each land use. While one might think of cropland under some land uses as an error, there are many cases where the back of a parcel is still used for agricultural purposes even though the dominant portion of the parcel is used for residential (as an example). Typically, the larger the parcel adjacent to agricultural lands the more we see this overlap. Of course some of the classification is error.

In [ ]:
lulc_crop_soilC <- c(70, 70, 70, 70, 70, 70, 70, 70, 70, 0, 70)

Lets put all these data together now using the `cbind()` function to derive a table of Mg C ha<sup>-1</sup> in soil by land use.

In [ ]:
soilCbylu <- cbind(lulc_categories, lulc_grass_soilC, lulc_trees_soilC, lulc_crop_soilC)

In [ ]:
# Mg C ha-1
soilCbylu <- as.data.frame(soilCbylu)
soilCbylu$lulc_grass_soilC <- as.numeric(soilCbylu$lulc_grass_soilC)
soilCbylu$lulc_trees_soilC <- as.numeric(soilCbylu$lulc_trees_soilC)
soilCbylu$lulc_crop_soilC <- as.numeric(soilCbylu$lulc_crop_soilC)

Lastly, lets give the table some column names and display it.

In [ ]:
# Mg C ha
colnames(soilCbylu) <- c("Land.Use","Grass","Trees","Crop")
soilCbylu

I always find it useful to also look at the results in kg C m<sup>-2</sup>, so lets just create the same table with those units.

In [ ]:
soilCbylukgm <- cbind(lulc_categories, soilCbylu$Grass * 1000 / 10000, soilCbylu$Trees * 1000 / 10000, soilCbylu$Crop * 1000 / 10000)

In [ ]:
# kg C m2
colnames(soilCbylukgm) <- c("Land.Use","Grass","Trees","Crop")
soilCbylukgm <- as.data.frame(soilCbylukgm)
soilCbylukgm$Grass <- round(as.numeric(soilCbylukgm$Grass), 2)
soilCbylukgm$Trees <- round(as.numeric(soilCbylukgm$Trees), 2)
soilCbylukgm$Crop <- round(as.numeric(soilCbylukgm$Crop), 2)
soilCbylukgm

# Estimates of Carbon by Land Use

Now that we have calculated estimates of soil C under grass, trees, and crop as well as an above-ground estimate of tree carbon, we can apply these values to a study area.
The Kitchener-Cambridge-Waterloo (KCW) Census Metropolitan Area (CMA) has been classified at 20 cm resolution in urban core areas and 80 cm in rural areas within the CMA boundary to derive the following land cover areas. The land use classification is the result of individual parcel manual interpretation. 

Read in a `.csv` of the area in different land covers using the `read.csv()` function.

In [ ]:
lc_data <- read.csv("landcoverarea.csv")

Lets take a quick look at these data.

In [ ]:
lc_data

Create a total area for each land use, by using the `rowSums()` function, and the percent of all area represented by each land use. We will also round our data to two decimals using the `round()` function.

In [ ]:
lc_data[ ,"Total"] <- rowSums(lc_data[,2:7])

Use the same approach to calculate the total percent of the study area represented by each land use type.

In [ ]:
lc_data[ ,"Perc"] <- round(lc_data$Total/sum(lc_data$Total)*100, 1)

Review these additions to our table of area (ha) of land cover by land use type.

In [ ]:
lc_data

Now we can calculate the carbon for each of the different land covers for each land use. To do this we will use a simple equation of <br>
<i>C<sub>Grass</sub> = LU<sub>Area</sub> * C<sub>Soils</sub></i> <br>
<i>C<sub>Crop</sub> = LU<sub>Area</sub> * C<sub>Soils</sub></i> <br>
<i>C<sub>Trees</sub> = LU<sub>Area</sub> * (C<sub>Soils</sub> + C<sub>Trees</sub>)</i><br>

we then divide the outcome by 1000 to convert from megagram (Mg) to gigagram (Gg) and we use the `round()` function to round the result to 2 decimal places. 

In [ ]:
C_grass <- round(lc_data$Grass * (soilCbylu$Grass)/1000, 2)

In [ ]:
C_crop <- round(lc_data$Crop * (soilCbylu$Crop)/1000, 2) 

In [ ]:
C_tree <- round(lc_data$Tree * (soilCbylu$Tree + C_trees_t_ha) / 1000, 2)

The result is a matrix that we convert to a dataframe using the `as.data.frame()` function. We assign column headings using the `colnames()` function and convert the columns to `numeric` data types instead of chr type.

In [ ]:
totalCarbon <- as.data.frame(cbind(lulc_categories, C_grass, C_tree, C_crop))
colnames(totalCarbon) <- c("Land.Use","Grass","Trees","Crop")
totalCarbon$Grass <- as.numeric(totalCarbon$Grass)
totalCarbon$Trees <- as.numeric(totalCarbon$Trees)
totalCarbon$Crop <- as.numeric(totalCarbon$Crop)

Next we calculate the total carbon for each land use type using `rowSums()` and the percent of all carbon stored in the landscape represented by each land use type.

In [ ]:
totalCarbon[ ,"Total"] <- rowSums(totalCarbon[,2:4])
totalCarbon[ ,"Perc"] <- round(totalCarbon$Total/sum(totalCarbon$Total)*100, 1)

Calculate the carbon density for each land use type.

In [ ]:
totalCarbon[ ,"Cdens"] <- round(totalCarbon$Total / lc_data$Total, 4)

Lastly, we can display our table of gigagrams (Gg) of carbon per land cover by land use type, with `Cdens` displaying Gg C ha<sup>-1</sup>

In [ ]:
totalCarbon

To better understand our carbon density values, we translate the density from Gg C ha<sup>-1</sup> to kg C m<sup>-2</sup>.

In [ ]:
# Gg / ha --> kg / m2
totalCarbon$Cdens * 100

# Estimates of Carbon by Parcel Size

Using the same classified imagery as above, areas of different land cover have been summarized by parcel size (ha) and put into a comma separated value (.csv) file. Next we'll read in that .csv and estimate carbon values for each land cover to see if there is a relationship between parcel size and carbon storage.

In [ ]:
lc_parcel_data <- read.csv("parcelha.csv")

#p.s., I'm getting a warning message here, which I can't seem to eliminate at the moment, but it is not causing any errors in subsequent calculations.

In [ ]:
lc_parcel_data

In [ ]:
lc_parcel_data[ ,"Total"] <- rowSums(lc_parcel_data[,2:7])

In [ ]:
lc_parcel_data[ ,"Perc"] <- round(lc_parcel_data$Total/sum(lc_parcel_data$Total)*100, 1)

In [ ]:
lc_parcel_data

The parcel data summary information includes all land-use types. Therefore, the carbon estimation uses an average soil carbon for Grass, Tree, and Crop from the soil carbon table for all land uses applied against the areas of each land cover within the different parcel size classes. Then we divide the outcome by 1000 to convert from megagram (Mg) to Gigagram (Gg) and use the `round()` function to round the result to 2 decimal places. 

In [ ]:
C_grass_parcel <- round(lc_parcel_data$Grass * (ave(soilCbylu$Grass)[1])/1000, 2)
C_crop_parcel <- round(lc_parcel_data$Crop * (ave(soilCbylu$Crop)[1])/1000, 2)
C_tree_parcel <- round(lc_parcel_data$Tree * (ave(soilCbylu$Tree)[1] + C_trees_t_ha) / 1000, 2)

As earlier, we can combine our parcel sizes with the carbon values we just calculated. Then ensure each column has a name and is a numeric data type (dbl).

In [ ]:
parcelCarbon <- as.data.frame(cbind(lc_parcel_data$ParcelSizeha, C_grass_parcel, C_tree_parcel, C_crop_parcel))
colnames(parcelCarbon) <- c("ParcelSizeha","Grass","Trees","Crop")
parcelCarbon$Grass <- as.numeric(parcelCarbon$Grass)
parcelCarbon$Trees <- as.numeric(parcelCarbon$Trees)
parcelCarbon$Crop <- as.numeric(parcelCarbon$Crop)

Now lets summarize the data by adding columns with the total amount of carbon, the percent of all parcel carbon represented by that parcel-size class, and the carbon density for that parcel size.

In [ ]:
parcelCarbon[ ,"Total"] <- rowSums(parcelCarbon[,2:4])

In [ ]:
parcelCarbon[ ,"Perc"] <- round(parcelCarbon$Total/sum(parcelCarbon$Total)*100, 1)

In [ ]:
parcelCarbon[ ,"Cdens"] <- round(parcelCarbon$Total / lc_parcel_data$Total, 4)

Then view the newly created table (i.e., dataframe).

In [ ]:
parcelCarbon

Again, I like to view some of the data in kg m<sup>-2</sup>. So lets just look at the densities that way.

In [ ]:
# Gg / ha --> kg / m-2
parcelCarbon$Cdens * 100

## Congratulations! You've completed this Notebook.